In [1]:
!pip install -U transformers wandb pandas matplotlib scikit-learn albumentations --quiet

# Import modules and load data

In [2]:
#autoload modules in notebook
%load_ext autoreload
%autoreload 2  

In [3]:
from itertools import combinations
import json
import os
import shutil
import pandas as pd
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import wandb
from transformers import DistilBertTokenizer, TFDistilBertModel
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import CLIP_model
import CLIP_data_load

2023-06-01 18:27:55.646504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 18:27:56.185335: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Hyperparameters
train_percentage = 0.8
test_percentage = 0.2
validation_percentage = 0.15
latent_dim_imgs = 1024
latent_dim_text = 768
latent_dim_common = 512
batch_size = 8
SEED = 116
img_shape = (128,128,3)
text_input_shape = (200)
bert_model_name = 'distilbert-base-uncased' #bert-base-multilingual-uncased
max_len = 200
model_name = 'clip_duplicated_text'

# Paths
all_images_path = 'dataset/resized_train/'
project_location = '/home/ale/Documents/University_arc/University/MSc/Transformers/project'
all_captions_path = f'{project_location}/dataset/caption_prediction_train.csv'  
all_concept_ids_path = f'{project_location}/dataset/concept_detection_train.csv'
all_concepts_path = f'{project_location}/dataset/concepts.csv'

zip_dataset_location = f'{project_location}/dataset/resized_train.zip'
dataset_extract_location = "/"
model_loc = f'{project_location}/weights/{model_name}.h5'

In [ ]:
run = wandb.init(project="clip",
           entity='calonca',
           name=model_name,
           config = {
              'batch_size':batch_size,
              'latent_dim_common':latent_dim_common,
              'latent_dim_text':latent_dim_text,
              'latent_dim_imgs':latent_dim_imgs,
              'img_shape':img_shape,
              'text_input_shape':text_input_shape,
              'model_name': model_name,
              'bert_model_name': bert_model_name,
           })

## **PRE TRAINED BERT MODEL**

## **PRE TRAINED FEATURE EXTRACTION NETWORK**

## **CLIP MODEL**

# Data preparation

In [7]:
#Merging all dataframes toghether into ID, caption, cuis, concepts
captionsDF = pd.read_csv(all_captions_path, sep='\t')
concept_id_df = pd.read_csv(all_concept_ids_path, sep='\t')
concept_df = pd.read_csv(all_concepts_path, sep=',')

concepts = concept_df.set_index('concept').T.to_dict('list')
concept_id_df.cuis = concept_id_df.cuis.apply(lambda x: x.split(';'))
concept_id_df['concepts'] = concept_id_df.cuis.apply(lambda cuis: {concepts[cui][0] for cui in cuis})

captionsDF = pd.merge(captionsDF,concept_id_df, on="ID")
captionsDF.ID = captionsDF.ID.apply(lambda x: x+'.jpg')

# **Data generation**

In [8]:
train_val, test = train_test_split(captionsDF, test_size = test_percentage, shuffle = True, random_state = SEED)
train, val = train_test_split(train_val, test_size = validation_percentage, shuffle = True, random_state = SEED)

In [12]:
train = CLIP_data_load.concepts_to_captions_append(train)
val = CLIP_data_load.concepts_to_captions_append(val)

In [18]:
train.iloc[99,:]['caption']

'density, praziquantel, physiologic resolution, Plain x-ray, Nodular (qualifier value), Chestchest radiograph show resolution of nodular density after oral praziquantel therapy'

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wand

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)

trainList = CLIP_data_load.paths_captions_emb_list(train,  all_images_path,tokenizer=tokenizer,max_len=max_len)
valList = CLIP_data_load.paths_captions_emb_list(val, all_images_path,tokenizer=tokenizer,max_len=max_len)

100%|██████████| 57720/57720 [00:10<00:00, 5360.71it/s]


In [ ]:
train_gen = CLIP_data_load.ClipBaseGenerator(data = trainList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)
val_gen = CLIP_data_load.ClipBaseGenerator(data = valList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)

## **Train CLIP**

In [ ]:
fen_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape = img_shape
)

bert_model = TFDistilBertModel.from_pretrained(bert_model_name)

model = CLIP_model.get_clip_model(
    image_input_shape = img_shape,
    text_input_shape = text_input_shape,
    text_encoder = bert_model,
    image_encoder = fen_model,
    latent_dim_imgs = latent_dim_imgs,
    latent_dim_text = latent_dim_text,
    latent_dim_common = latent_dim_common,
    train_bert = True
)

2023-06-01 18:20:42.958751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-01 18:20:43.116022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-01 18:20:43.116234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

82420632/82420632 [==============================] - 3s 0us/step


2023-06-01 18:20:51.407598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of 

In [ ]:
model.summary()

Model: "clip_base"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_3[0][0]',        

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5,restore_best_weights=True)

In [ ]:
from wandb.keras import WandbCallback
wandb_callback = WandbCallback(monitor='loss',
                               log_batch_frequency=10,
                               save_model = False,
                               validation_steps=5)


In [ ]:
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 100, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

In [ ]:
!mkdir -p {project_location}/weights
model.save(f"{project_location}/weights/{model_name}.h5")

In [ ]:
artifact = wandb.Artifact(model_name, type='model')
artifact.add_file(f"{project_location}/weights/{model_name}.h5")
run.log_artifact(artifact)
run.finish()